In [22]:
import os
os.environ["DASHSCOPE_API_KEY"] = "sk-5e56b4e6ddba4a6a8110e168d8dd8377"

os.environ["TAVILY_API_KEY"] ='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'

In [23]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()
 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS

from langchain.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import Chroma


embeddings = DashScopeEmbeddings(
    model="text-embedding-v1"
)

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()
 
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [24]:
from langchain_community.llms import Tongyi

tools = [search, retriever_tool]
llm = Tongyi()
 
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent") # Get the prompt to use
 
from langchain.agents import create_openai_functions_agent
agent = create_openai_functions_agent(llm, tools, prompt)
 
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [25]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...


TypeError: Additional kwargs key output_tokens already exists in left dict and value has unsupported type <class 'int'>.

In [ ]:
!pip install langchainhub -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://pypi.ngc.nvidia.com
